In [1]:
# TDO COMBINE TRAIN HISTORY DATA TO ONE FILE PER DAY
# TODO CALC TRAVALED PASSENGERN FROM TO FOR EACH DAY FOR EVERY STATION TO ONE! NEXT STATION

In [2]:
import os
import pandas as pd
from pathlib import Path
from datetime import datetime



cwd = os.getcwd()
dep_table_path = os.path.join(cwd, 'generated/departure_tables_raw')
dep_table_path

'/home/jovyan/work/generated/departure_tables_raw'

In [3]:
raw_departure_tables = []
departure_tables_dates = []
departure_tables_dates_ts = []
# LOAD ALL CSV FILES INTO ONE BIG CHUNK
files = [ f for f in os.listdir( dep_table_path)] #list comprehension version.

for f in files:
    if(Path(f).suffix == ".csv"):
        filename = str(Path(f).stem)
        if '_' in filename and len(filename.split('_')) == 4:
            # THE GOAL IS TO COMBINE ALL TRAIN DEPARTURES FROM THE SAME DAY
            # SO THE FILE TIMESTAMP IN ITS NAME CORRESPDS TO THE DATE OF RECORD = TRAIN DEPARTURES
            # BY ADDING THE NEW DATE TO THE PD DF ITS IS POSSIBLE TO SORT FOR DATES LATER ON
            timestamp = int(filename.split('_')[3])
            date = datetime.fromtimestamp(timestamp)
            # THIS WILL BE ADDED TO THE DF AS NEW COLUMN AFTER LOADING
            combined_day = int(str(date.day) + str(date.month) + str(date.year))
            departure_tables_dates.append(combined_day)
            departure_tables_dates_ts.append(timestamp)
            # LOAD FILE TO DF
            abs_path = os.path.join(dep_table_path,f)
            df = pd.read_csv(abs_path,sep=';',encoding="utf-8",dtype = str)
            
            # ADD NEW COLUMNS
            df['date'] = pd.Series([date for x in range(len(df.index))])
            df['combined_date'] = pd.Series([combined_day for x in range(len(df.index))])
            
            raw_departure_tables.append(df)


            
# CREATE DATASET FROM DATE VALUES
df_departure_tables_dates = pd.DataFrame(data={'recorded_days':departure_tables_dates,'timestamp':departure_tables_dates_ts})
            
# AFER LOADING ALL RAW FILE IN CREATE ONE GIANT DATASET
departures_combined = pd.concat(raw_departure_tables)
departures_combined.head(5)


,current_station_departure_time,current_station_name,type,name,line,number,current_station,stops,max_capacity,date,combined_date,start_station
0,2021-11-29T19:29:00.000Z,Bayreuth Hbf,RE,RE 30,30,3092,8000028,"8000028%2021-11-29T19:35:00.000Z%6,8004759,800...",602,2021-11-29 19:33:41,29112021,NaN
1,2021-11-29T20:03:00.000Z,Bayreuth Hbf,RE,RE 38,38,59316,8000028,"8000028%2021-11-29T20:03:00.000Z%0,8005895,800...",602,2021-11-29 19:33:41,29112021,NaN
2,2021-11-29T20:13:00.000Z,Bayreuth Hbf,RE,RE 32,32,3418,8000028,"8000028%2021-11-29T20:13:00.000Z%0,8001348,800...",602,2021-11-29 19:33:41,29112021,NaN
3,2021-11-29T21:05:00.000Z,Bayreuth Hbf,RE,RE 32,32,3447,8000028,"8000028%2021-11-29T21:05:00.000Z%,8000974,8004...",602,2021-11-29 19:33:41,29112021,NaN
4,2021-11-29T21:27:00.000Z,Bayreuth Hbf,RE,RE 30,30,3094,8000028,"8000028%2021-11-29T21:27:00.000Z%,8001348,8000...",602,2021-11-29 19:33:41,29112021,NaN


In [4]:
# REMOVE DUPLICATE TRAINS TRAIN_ID AND START_STATION
departures_combined = departures_combined.drop_duplicates(['number','start_station','combined_date'],keep= 'last')
departures_combined.head(5)


,current_station_departure_time,current_station_name,type,name,line,number,current_station,stops,max_capacity,date,combined_date,start_station
17,2021-11-29T19:15:00.000Z,Bielefeld-Brackwede,RB,RB 67,67,79317,8000048,"8000048%2021-11-29T19:30:00.000Z%15,8002461,80...",426,2021-11-29 19:33:41,29112021,NaN
129,2021-11-29T19:30:00.000Z,Landshut(Bay)Hbf,RE,RE 50,50,4866,8000217,"8000217%2021-11-29T19:32:00.000Z%2,8001835,800...",602,2021-11-29 19:33:41,29112021,NaN
779,2021-11-29T19:29:00.000Z,Hof Hbf,RE,RE 3,3,74035,8002924,"8002924%2021-11-29T19:29:00.000Z%0,8010275,801...",602,2021-11-29 19:33:41,29112021,NaN
982,2021-11-29T19:30:00.000Z,Koblenz-Moselweiß,RB,RB 81,81,21597,8003354,"8003354%2021-11-29T19:30:00.000Z%0,8000206,",426,2021-11-29 19:33:41,29112021,NaN
2174,2021-11-29T19:30:00.000Z,Oker,RB,RB 82,82,14134,8004665,"8004665%2021-11-29T19:30:00.000Z%0,8000130,800...",426,2021-11-29 19:33:41,29112021,NaN


In [5]:
# LETS TEST THE DATA
# GET ALL TRAIN DEPARTURES FROM Bayreuth Hbf [8000028] @ 29.11.2021
# RV + FV + DB ONLY
tmp = departures_combined.loc[departures_combined['start_station'] == str(8000028)]
tmp = tmp.loc[tmp['combined_date'] == departure_tables_dates[0]]
tmp = tmp.reset_index()
tmp.head(100)

,index,current_station_departure_time,current_station_name,type,name,line,number,current_station,stops,max_capacity,date,combined_date,start_station
0,10,NaN,NaN,RE,RE 38,38,59310,NaN,"8000028,8005895,8000267,8006017,8003476,800128...",602,2021-11-29 12:41:28,29112021,8000028
1,11,NaN,NaN,RE,RE 32,32,3464,NaN,"8000028,8004759,8002794,8000284,",602,2021-11-29 12:41:28,29112021,8000028
2,12,NaN,NaN,RE,RE 30,30,3414,NaN,"8000028,8001348,8000328,8004759,8004284,8000284,",602,2021-11-29 12:41:28,29112021,8000028
3,0,NaN,NaN,RE,RE 30,30,3091,NaN,"8000028,8004126,8002924,",602,2021-11-29 16:47:02,29112021,8000028
4,1,NaN,NaN,RE,RE 32,32,3476,NaN,"8000028,8001348,8004759,8002794,8000284,",602,2021-11-29 16:47:02,29112021,8000028
5,2,NaN,NaN,RE,RE 32,32,59312,NaN,"8000028,8005895,8000267,8003476,8003813,800128...",602,2021-11-29 16:47:02,29112021,8000028
6,7,NaN,NaN,RE,RE 38,38,59308,NaN,"8000028,8005895,8000267,8006017,8003476,800128...",602,2021-11-29 11:40:48,29112021,8000028
7,8,NaN,NaN,RE,RE 32,32,3462,NaN,"8000028,8004759,8002794,8000284,",602,2021-11-29 11:40:48,29112021,8000028
8,9,NaN,NaN,RE,RE 30,30,3412,NaN,"8000028,8001348,8000328,8004759,8004284,8000284,",602,2021-11-29 11:40:48,29112021,8000028
9,10,NaN,NaN,RE,RE 30,30,3089,NaN,"8000028,8004126,8002924,",602,2021-11-29 11:40:48,29112021,8000028


In [6]:
# COUNT THE UNIQUE TRAINS
#THE TRAIN_ID IS UNIQUE FOR EACH TRAIN AND IDENTIFIES THE TRAIN OVER ITS COMPLETE COURSE
# ONE UNIQUE ID = ONE ZUGLAUF
# FROM START TO END-STATION
tmp = departures_combined.loc[departures_combined['combined_date'] == departure_tables_dates[0]]
tmp['number'].nunique()

7832

In [7]:
# SAVE RESULTS
departures_combined.to_csv("./generated/4_departure_table.csv",sep=';',encoding="utf-8",index=False)
df_departure_tables_dates.to_csv("./generated/4_departure_table_day_list.csv",sep=';',encoding="utf-8",index=False)